In [233]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [234]:
PROBLEM_SIZE = 20
NUM_SETS = 40
np.random.seed(42)
SETS = tuple(
    np.array([np.random.rand() < 0.2 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple('State', ['taken', 'not_taken'])

In [235]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))


def g(state):
    #print("g(n)",len(state.taken))
    return len(state.taken)
#The heuristic value should represent a lower bound for the cost.
#I thought to consider for a given state the unused sets and for each of them considering
#what is the improvement level. In other words, how many uncovered elements they are able to cover.
#The uncovered elements are the only things that still matter.
#At this point, sorting the unused sets based on their improvement level and considering
#as cost lower bound how many sets I need to consider such that the sum of their.
#improvement level is >= uncovered elements.
#EXAMPLE
#If I have 3 elements still uncovered, and 4 sets unused with improvements level equale to:
#1,1,2,1. For sure I need to consider at LEAST 2 sets. I am sure that it is impossible to consider 
#just one set since no set has 3 as improvement level.

def h(state):
    
    sorted_l = []
    level_impro = []  #list that contain the levels of improvement
    cost_lb = 0       #lower bound provided by the heuristic
    sum_impro = 0     #sum of the level of impro
    
    actual_situation = reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]))
    uncovered_elements = PROBLEM_SIZE - sum(actual_situation) #number of uncovered elements
    
    for i in state.not_taken:
        impro = np.sum(SETS[i] &~ actual_situation)    #computing how many uncovered elements that given set covers
        level_impro.append(impro)
    
    sorted_l = sorted(level_impro, reverse=True)
    #print(sorted_l)
    
    for l in sorted_l: 
        if sum_impro < uncovered_elements:
            sum_impro += l
            cost_lb +=1
        else:
            #print('h(n):', cost_lb)
            return cost_lb
    
            

def f(state):
    #print("f(n):",g(state) + h(state))
    return  g(state) + h(state)


In [236]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Problem not solvable"

In [237]:
frontier = PriorityQueue()
#frontier = SimpleQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))
print("Initial State ->",state)
counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    print("Current State:", current_state)
    counter += 1
    for action in current_state[1]: #the idea is to sort the action, it means ordering the sets to be 
        new_state = State(          #inserted based on ratio of overlapping (number of overlap/number of covered elements)
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
        print((f(new_state), new_state))
    
    print("---")    
    _, current_state = frontier.get()
    print("Primo della frontiera:",current_state)
print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

g(n) 0
h(n) 2
f(n): 2
g(n) 0
h(n) 2
State(taken=set(), not_taken={0, 1, 2, 3, 4})
g(n) 1
h(n) 2
f(n): 3
g(n) 1
h(n) 2
g(n) 1
h(n) 1
f(n): 2
g(n) 1
h(n) 1
g(n) 1
h(n) 1
f(n): 2
g(n) 1
h(n) 1
g(n) 1
h(n) 2
f(n): 3
g(n) 1
h(n) 2
g(n) 1
h(n) 1
f(n): 2
g(n) 1
h(n) 1
State(taken={1}, not_taken={0, 2, 3, 4})
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
State(taken={2}, not_taken={0, 1, 3, 4})
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
State(taken={4}, not_taken={0, 1, 2, 3})
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
State(taken={0}, not_taken={1, 2, 3, 4})
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 1
g(n) 2
h(n) 2
f(n): 4
g(n) 2
h(n) 2
g(n) 2
h(n) 1
f(n): 3
g(n) 2
h(n) 

In [238]:
current_state

State(taken={2, 4}, not_taken={0, 1, 3})